In [ ]:
!pip install pymorphy2
!pip install natasha
!pip install navec
!pip install slovnet

In [13]:
!pip install pyaspeller
from pyaspeller import YandexSpeller
speller = YandexSpeller()

In [75]:
import json
import re
import string
from pymorphy2.tokenizers import simple_word_tokenize
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()
from urllib.request import urlretrieve
import os.path
import pandas as pd

from navec import Navec
from slovnet import NER

destination = 'navec_news_v1_1B_250K_300d_100q.tar'
destination_2 = 'slovnet_ner_news_v1.tar'
if os.path.isfile(destination):
    pass
else:
    url = 'https://storage.yandexcloud.net/natasha-navec/packs/navec_news_v1_1B_250K_300d_100q.tar'
    url_2 = 'https://storage.yandexcloud.net/natasha-slovnet/packs/slovnet_ner_news_v1.tar'
    urlretrieve(url, destination)
    urlretrieve(url_2, destination_2)

navec = Navec.load('navec_news_v1_1B_250K_300d_100q.tar')
ner = NER.load('slovnet_ner_news_v1.tar')
ner.navec(navec)

from nltk.corpus import stopwords
stop_words = set(stopwords.words('russian'))


import requests
url = 'https://raw.githubusercontent.com/odaykhovskaya/obscene_words_ru/master/obscene_corpus.txt'
r = requests.get(url)
obscene_words = r.text.lower().split('\n')[:-1]

# preprocessing

In [85]:
!python -m pip install emoji --upgrade
import emoji
import nltk
nltk.download('stopwords')

Requirement already up-to-date: emoji in /usr/local/lib/python3.7/dist-packages (1.2.0)
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

проверка орфографии

In [16]:
def spel(text):
  changes = {}
  text = re.sub('\[id\d*\|\w*\]', 'username', text)
  for change in speller.spell(text):
    # print(change['s'][0])
    if change['s'] != []:
      changes[change['word']] = change['s'][0]
  for word, suggestion in changes.items():
    text = text.replace(word, suggestion)
  return text

создание словаря со всеми вариантами препроцессинга

In [10]:
def make_meta():
    meta = {}
    i = 0
    for punct in ['no', 'yes']:
        for ner in ['no', 'del', 'replace']:
            for lem in ['no', 'yes']:
                for stop in ['no', 'yes']:
                    for emo in ['no', 'del', 'replace', 'label']:
                        for vulg in ['no', 'yes']:
                            meta[i] = {'punctuation_deletion': punct,
                                       'ner_processing': ner,
                                       'lemmatization': lem,
                                       'stopwords_deletion': stop,
                                       'emojis_processing': emo,
                                       'vulgar_processing': vulg}
                            i += 1

    with open('meta.json', 'w') as f:
        json.dump(meta, f, ensure_ascii=False)

    return meta

In [105]:
meta = make_meta()

In [93]:
train = pd.read_excel('/content/data_train.xlsx', usecols = ['текст',	'контекст', 'index'])[:5]
test = pd.read_excel('/content/data_test.xlsx', usecols = ['текст',	'контекст', 'index'])[:5]

In [94]:
with open ('/content/emoji_list.json') as f:
  list_emojis = json.load(f)

In [97]:
def tokenize(text: str):
  sentence = text.lower()
  sentence = re.sub('\[id\d*\|\w*\]', 'username', sentence)
  tokens = simple_word_tokenize(sentence)
  tokens = ' '.join(tokens)
  tokens = re.sub('ё', 'е', tokens)
  tokens = re.sub(r'(?<=\w)\*', 'ё', tokens)
  return tokens

In [98]:
def lemmatize(text: str):
  tokens = text.split()
  tokens = [morph.parse(i)[0].normal_form for i in tokens]
  tokens = " ".join(tokens)
  return tokens

In [99]:
def ner_process(text, parametr):
    if parametr == 'no':
        return text
    try:
        markup = ner(text).spans
        for spans in markup:
            ner_word = text[spans.start:spans.stop]
            if parametr == 'del':
                text = re.sub(ner_word, '', text)
            elif parametr == 'replace':
                text = re.sub(ner_word, ' NER ', text)
            else:
                raise ValueError('такой опции нет')
            text = " ".join(text.split())
    except Exception:
        pass
    return text

In [100]:
def preprocess(text, params={'punctuation_deletion': 'no',
                             'ner_processing': 'no',
                             'lemmatization': 'no',
                             'stopwords_deletion': 'no', 
                             'emojis_processing': 'no', 
                             'vulgar_processing': 'no'}):
    
    if params['punctuation_deletion'] == 'no':
        pass 
    elif params['punctuation_deletion'] == 'yes':
        text = text.split()
        text = [i for i in text if (i not in string.punctuation)]
        text = " ".join(text)
    else:
        raise ValueError('такой опции нет')
        
    text = ner_process(text, params['ner_processing'])
        
    if params['stopwords_deletion'] == 'no':
        pass 
    elif params['stopwords_deletion'] == 'yes':
        text = text.split()
        text = [t for t in text if t not in stop_words]
        text = " ".join(text)
    else:
        raise ValueError('такой опции нет')
        
        
    if params['emojis_processing'] == 'no':
        pass 
    elif params['emojis_processing'] == 'del':
          emo = emoji.demojize(text)
          text = re.sub(':.*?:', '', emo)
          text = re.sub('\s+', ' ', text)

    elif params['emojis_processing'] == 'replace':
          emo = emoji.demojize(text)
          text = re.sub(':.*?:', 'emoji', emo)
          text = re.sub('\s+', ' ', text)

    elif params['emojis_processing'] == 'label':
          text = text.split()
          new_tokens = []
          for t in text:
              if t in list_emojis[0]:
                  t = 'pos_emoji'
              elif t in list_emojis[1]:
                  t = 'neg_emoji'
              else:
                  t = re.sub(':.*?:', 'neutral_emoji', emoji.demojize(t))
              new_tokens.append(t)
          text = " ".join(new_tokens)
    else:
        raise ValueError('такой опции нет')
        
        
    if params['vulgar_processing'] == 'no':
        pass 
    elif params['vulgar_processing'] == 'yes':
        tokens = text.split()
        for i, v in enumerate(tokens):
            if 'ё' in v:
                tokens[i] = 'vulgar'
            elif v in obscene_words:
                tokens[i] = 'vulgar'
        text = ' '.join(tokens)
    else:
        raise ValueError('такой опции нет')
    return text

In [102]:
def make_preproc(data, meta):
    data['текст_спел'] = data['текст'].apply(spel)
    data['tokenized'] = data['текст_спел'].apply(tokenize)
    data['lemmatized'] = data['tokenized'].apply(lemmatize)

    for i, params in meta.items():
        if params['lemmatization'] == 'no':
            data[i] = [preprocess(tok, params=params) for tok in data['tokenized']]
        if params['lemmatization'] == 'yes':
            data[i] = [preprocess(tok, params=params) for tok in data['lemmatized']]
    return data

In [103]:
make_preproc(test, meta)

,текст,контекст,index,текст_спел,tokenized,lemmatized,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,...,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191
0,ты че несешь )) с какими мусульманами че там у...,"username, у нас также с мусульманами начнётся ...",1,ты че несешь )) с какими мусульманами че там у...,ты че несешь ) ) с какими мусульманами че там ...,ты че нести ) ) с какой мусульманин че там у т...,ты че несешь ) ) с какими мусульманами че там ...,ты че несешь ) ) с какими мусульманами че там ...,ты че несешь ) ) с какими мусульманами че там ...,ты че несешь ) ) с какими мусульманами че там ...,ты че несешь ) ) с какими мусульманами че там ...,ты че несешь ) ) с какими мусульманами че там ...,ты че несешь ) ) с какими мусульманами че там ...,ты че несешь ) ) с какими мусульманами че там ...,че несешь ) ) какими мусульманами че начнется ...,че несешь ) ) какими мусульманами че начнется ...,че несешь ) ) какими мусульманами че начнется ...,че несешь ) ) какими мусульманами че начнется ...,че несешь ) ) какими мусульманами че начнется ...,че несешь ) ) какими мусульманами че начнется ...,че несешь ) ) какими мусульманами че начнется ...,че несешь ) ) какими мусульманами че начнется ...,ты че нести ) ) с какой мусульманин че там у т...,ты че нести ) ) с какой мусульманин че там у т...,ты че нести ) ) с какой мусульманин че там у т...,ты че нести ) ) с какой мусульманин че там у т...,ты че нести ) ) с какой мусульманин че там у т...,ты че нести ) ) с какой мусульманин че там у т...,ты че нести ) ) с какой мусульманин че там у т...,ты че нести ) ) с какой мусульманин че там у т...,че нести ) ) мусульманин че начаться хахи ) ) ),че нести ) ) мусульманин че начаться хахи ) ) ),че нести ) ) мусульманин че начаться хахи ) ) ),че нести ) ) мусульманин че начаться хахи ) ) ),че нести ) ) мусульманин че начаться хахи ) ) ),че нести ) ) мусульманин че начаться хахи ) ) ),че нести ) ) мусульманин че начаться хахи ) ) ),че нести ) ) мусульманин че начаться хахи ) ) ),ты че несешь ) ) с какими мусульманами че там ...,ты че несешь ) ) с какими мусульманами че там ...,...,че нести мусульманин че начаться хахи,че нести мусульманин че начаться хахи,че нести мусульманин че начаться хахи,че нести мусульманин че начаться хахи,че нести мусульманин че начаться хахи,че нести мусульманин че начаться хахи,че нести мусульманин че начаться хахи,че нести мусульманин че начаться хахи,ты че несешь с какими мусульманами че там у те...,ты че несешь с какими мусульманами че там у те...,ты че несешь с какими мусульманами че там у те...,ты че несешь с какими мусульманами че там у те...,ты че несешь с какими мусульманами че там у те...,ты че несешь с какими мусульманами че там у те...,ты че несешь с какими мусульманами че там у те...,ты че несешь с какими мусульманами че там у те...,че несешь какими мусульманами че начнется хахах,че несешь какими мусульманами че начнется хахах,че несешь какими мусульманами че начнется хахах,че несешь какими мусульманами че начнется хахах,че несешь какими мусульманами че начнется хахах,че несешь какими мусульманами че начнется хахах,че несешь какими мусульманами че начнется хахах,че несешь какими мусульманами че начнется хахах,ты че нести с какой мусульманин че там у ты на...,ты че нести с какой мусульманин че там у ты на...,ты че нести с какой мусульманин че там у ты на...,ты че нести с какой мусульманин че там у ты на...,ты че нести с какой мусульманин че там у ты на...,ты че нести с какой мусульманин че там у ты на...,ты че нести с какой мусульманин че там у ты на...,ты че нести с какой мусульманин че там у ты на...,че нести мусульманин че начаться хахи,че нести мусульманин че начаться хахи,че нести мусульманин че начаться хахи,че нести мусульманин че начаться хахи,че нести мусульманин че начаться хахи,че нести мусульманин че начаться хахи,че нести мусульманин че начать

In [104]:
make_preproc(train, meta)

,текст,контекст,index,текст_спел,tokenized,lemmatized,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,...,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191
0,"username, о ""Северный поток""!!! Приди, спаси э...","нет, есть масса доказательств и телефонных раз...",1,"username, о ""Северный поток""!!! Приди, спаси э...","username , о "" северный поток "" ! ! ! приди , ...","username , о "" северный поток "" ! ! ! прийти ,...","username , о "" северный поток "" ! ! ! приди , ...","username , о "" северный поток "" ! ! ! приди , ...","username , о "" северный поток "" ! ! ! приди , ...","username , о "" северный поток "" ! ! ! приди , ...","username , о "" северный поток "" ! ! ! приди , ...","username , о "" северный поток "" ! ! ! приди , ...","username , о "" северный поток "" ! ! ! приди , ...","username , о "" северный поток "" ! ! ! приди , ...","username , "" северный поток "" ! ! ! приди , сп...","username , "" северный поток "" ! ! ! приди , сп...","username , "" северный поток "" ! ! ! приди , сп...","username , "" северный поток "" ! ! ! приди , сп...","username , "" северный поток "" ! ! ! приди , сп...","username , "" северный поток "" ! ! ! приди , сп...","username , "" северный поток "" ! ! ! приди , сп...","username , "" северный поток "" ! ! ! приди , сп...","username , о "" северный поток "" ! ! ! прийти ,...","username , о "" северный поток "" ! ! ! прийти ,...","username , о "" северный поток "" ! ! ! прийти ,...","username , о "" северный поток "" ! ! ! прийти ,...","username , о "" северный поток "" ! ! ! прийти ,...","username , о "" северный поток "" ! ! ! прийти ,...","username , о "" северный поток "" ! ! ! прийти ,...","username , о "" северный поток "" ! ! ! прийти ,...","username , "" северный поток "" ! ! ! прийти , с...","username , "" северный поток "" ! ! ! прийти , с...","username , "" северный поток "" ! ! ! прийти , с...","username , "" северный поток "" ! ! ! прийти , с...","username , "" северный поток "" ! ! ! прийти , с...","username , "" северный поток "" ! ! ! прийти , с...","username , "" северный поток "" ! ! ! прийти , с...","username , "" северный поток "" ! ! ! прийти , с...","username , о "" северный поток "" ! ! ! приди , ...","username , о "" северный поток "" ! ! ! приди , ...",...,username северный поток прийти спасти экономик...,username северный поток прийти спасти экономик...,username северный поток прийти спасти экономик...,username северный поток прийти спасти экономик...,username северный поток прийти спасти экономик...,username северный поток прийти спасти экономик...,username северный поток прийти спасти экономик...,username северный поток прийти спасти экономик...,username о северный поток приди спаси экономик...,username о северный поток приди спаси экономик...,username о северный поток приди спаси экономик...,username о северный поток приди спаси экономик...,username о северный поток приди спаси экономик...,username о северный поток приди спаси экономик...,username о северный поток приди спаси экономик...,username о северный поток приди спаси экономик...,username северный поток приди спаси экономику ...,username северный поток приди спаси экономику ...,username северный поток приди спаси экономику ...,username северный поток приди спаси экономику ...,username северный поток приди спаси экономику ...,username северный поток приди спаси экономику ...,username северный поток приди спаси экономику ...,username северный поток приди спаси экономику ...,username о северный поток прийти спасти эконом...,username о северный поток прийти спасти эконом...,username о северный поток прийти спасти эконом...,username о северный поток прийти спасти эконом...,username о северный поток прийти спасти эконом...,username о северный поток прийти спасти эконом...,username о северный поток прийти спасти эконом...,username о северный поток пр